Pobieramy dane. Alternatywnie z zapisanego pliku lokalnie.
Źródło danych: (GPW) https://www.gpw.pl/archiwum-notowan 

In [ ]:
import pandas as pd
import numpy as np

# get data
url = 'https://drive.google.com/file/d/1VfvaefBkfg0fvEkDwgBep10Vs3T5egWH/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]

df = pd.read_csv(path)
df['Data'] = pd.to_datetime(df['Data'])
df = df.set_index('Data')

Definicja funkcji RSI

In [ ]:
### by Jev //
def rsi(price, n=14):
    # calculate price gain with previous day, first row nan is filled with 0
    gain = (price-price.shift(1)).fillna(0) 
    def rsiCalc(p):
        # subfunction for calculating rsi for one lookback period
        avgGain = p[p>0].sum()/n
        avgLoss = -p[p<0].sum()/n 
        rs = avgGain/avgLoss
        return 100 - 100/(1+rs)

    # run for all periods with rolling_apply
    return gain.rolling(n).apply(rsiCalc) 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec 

mask = df.index > '2020-01-01'
df['rsi'] = rsi(df['close'])

# create grid for different subplots
spec = gridspec.GridSpec(ncols=1, nrows=2, width_ratios=[1],wspace=0.5,hspace=0.2, height_ratios=[3,1])

# create a figure 
fig = plt.figure(figsize=[10,10]) 
ax0 = fig.add_subplot(spec[0]) 
ax1 = fig.add_subplot(spec[1]) 
ax1.set_yticks([30, 70])
ax0.set(title='CLOSE CDR')
ax1.set(title='RSI len=14')

ax0.plot(df['close'][mask])
ax1.plot(df['rsi'][mask])
ax1.yaxis.grid(True)

Przykładowa strategia RSI
RSI rozne dlugosci / 30 / 70

In [ ]:
for l in range(10, 20, 1):
    dr = df[df.index > '2019-01-01']
    dr['rsi'] = rsi(dr['close'], l)

    positive_mask = (dr['rsi']>30) & (dr['rsi']<70) 
    buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
    close_buy_mask = positive_mask & ~positive_mask.shift(periods=-1).fillna(False)

    #gain %
    gain = np.sum((dr['close'].loc[close_buy_mask].to_numpy()-dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)])/dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)])

    print('rsi len:%d  gain:%.2f%%  num_trans:%d' %(l, 100*gain, np.sum(close_buy_mask)))

Strategia: kupuj gdy krus przecina srednia o dlugosci len.

In [ ]:
dr = df[df.index > '2019-01-01']

for l in range(10, 100, 10):

    dr['avg'] = df['close'].rolling(window=l).mean()[df.index > '2019-01-01']

    positive_mask = (dr['close'] > dr['avg']) 
    buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
    close_buy_mask = ~positive_mask & positive_mask.shift(periods=1).fillna(False)

    #gain %
    gain = np.sum(dr['close'].loc[close_buy_mask].to_numpy() / dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)] - 1)

    print('len:%d  gain:%.2f%%  num_trans:%d' %(l, 100*gain, np.sum(close_buy_mask)))

Znajdz 2 srednie, ktorych przeciecie daloby najlepsze rezultaty

In [ ]:
dr = df[df.index > '2019-01-01']
maxgain = 0
for len_slow in range(10, 100, 10):
    for len_fast in range(5, 100, 5):
        if len_fast >= len_slow:
            continue

        dr['avg_fast'] = df['close'].rolling(window=len_fast).mean()[df.index > '2019-01-01']
        dr['avg_slow'] = df['close'].rolling(window=len_slow).mean()[df.index > '2019-01-01']

        positive_mask = (dr['avg_fast'] > dr['avg_slow']) 
        buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
        close_buy_mask = ~positive_mask & positive_mask.shift(periods=1).fillna(False)

        #gain %
        gain = np.sum(dr['close'].loc[close_buy_mask].to_numpy() / dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)] - 1)
        if gain > maxgain:
            maxgain = gain
            print('New max! len_slow:%d  len_fast:%d  gain:%.2f%%  num_trans:%d' %(len_slow, len_fast, 100*gain, np.sum(close_buy_mask)))

wykres + transkacje dla  len_slow:60 len_fast:40

In [ ]:
maxgain = 0
len_fast = 40
len_slow = 60
dr['avg_fast'] = df['close'].rolling(window=len_fast).mean()[df.index > '2019-01-01']
dr['avg_slow'] = df['close'].rolling(window=len_slow).mean()[df.index > '2019-01-01']

positive_mask = (dr['avg_fast'] > dr['avg_slow']) 
buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
close_buy_mask = ~positive_mask & positive_mask.shift(periods=1).fillna(False)

#gain %
gain = np.sum(dr['close'].loc[close_buy_mask].to_numpy() / dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)] - 1)
if gain > maxgain:
    maxgain = gain
    print('New max! len_slow:%d  len_fast:%d  gain:%.2f%%  num_trans:%d' %(len_slow, len_fast, 100*gain, np.sum(close_buy_mask)))

#wykres
tmp = pd.concat([dr[['close', 'avg_fast', 'avg_slow']], pd.DataFrame(buy_mask, columns=['buy']).replace({False:np.nan, True:1}), pd.DataFrame(close_buy_mask, columns=['close_buy']).replace({False:np.nan, True:1})], axis=1)
tmp['buy'] = tmp['buy'] * tmp['close']
tmp['close_buy'] = tmp['close_buy'] * tmp['close']
tmp[['close', 'avg_fast', 'avg_slow']].plot()
tmp['close_buy'].plot(style='or')
tmp['buy'].plot(style='og')    

In [ ]:
# transakcje LONG
newdf = pd.DataFrame(tmp['buy'][~tmp['buy'].isna()])
newdf['close_buy'] = tmp['close_buy'][~tmp['close_buy'].isna()].to_numpy()
newdf['gain%'] = round(100 * (newdf['close_buy'] / newdf['buy'] - 1), 2)
newdf['buy_date'] = tmp.index[~tmp['buy'].isna()]
newdf['close_date'] = tmp.index[~tmp['close_buy'].isna()]
newdf

SHORT

In [ ]:
negative_mask = (dr['avg_fast'] < dr['avg_slow']) 
sell_mask = negative_mask & ~negative_mask.shift(periods=1).fillna(False)
close_sell_mask = ~negative_mask & negative_mask.shift(periods=1).fillna(False)

#gain %
if np.sum(sell_mask) > np.sum(close_sell_mask):
    #sztucznie zamykamy ostatnią transkację ostatniego dnia
    close_sell_mask[-1] = True

gain = np.sum(-(dr['close'].loc[close_sell_mask].to_numpy() / dr['close'].loc[sell_mask].to_numpy() - 1))

print(100*gain)

In [ ]:
# transakcje SHORT
newdf = pd.DataFrame(tmp['close'][sell_mask])
newdf['close_sell'] = tmp['close'][close_sell_mask].to_numpy()
newdf['gain%'] = round(-100 * (newdf['close_sell'] / newdf['close'] - 1), 2)
newdf['sell_date'] = tmp.index[sell_mask]
newdf['close_date'] = tmp.index[close_sell_mask]
newdf

EMA

In [ ]:
dr = df[df.index > '2019-01-01']
maxgain = 0
for len_slow in range(10, 100, 10):
    for len_fast in range(5, 100, 5):
        if len_fast >= len_slow:
            continue

        dr['avg_fast'] = df['close'].ewm(halflife=len_fast).mean()[df.index > '2019-01-01']
        dr['avg_slow'] = df['close'].ewm(halflife=len_slow).mean()[df.index > '2019-01-01']

        positive_mask = (dr['avg_fast'] > dr['avg_slow']) 
        buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
        close_buy_mask = ~positive_mask & positive_mask.shift(periods=1).fillna(False)

        #gain %
        gain = np.sum(dr['close'].loc[close_buy_mask].to_numpy() / dr['close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)] - 1)
        if gain > maxgain:
            maxgain = gain
            print('New max! len_slow:%d  len_fast:%d  gain:%.2f%%  num_trans:%d' %(len_slow, len_fast, 100*gain, np.sum(close_buy_mask)))

Rozkład normalny przypadkowych zmian

In [ ]:
dr = df[df.index>'2019-01-01']

np.random.seed(1) #seed numpy to have same initialized weitghts
random_chng = np.random.normal(0, 0.1, dr.shape[0])

dr['rand_changes'] = random_chng
dr['rand_changes'].plot.hist(legend=True)

rozkład częstości zmian dla rzeczywistego kursu

In [ ]:
dr = df[df.index > '2019-01-01']
dr['close'].pct_change().fillna(0).plot.hist()

Sztucznie wygenerowany "kurs"

In [ ]:
random_close = np.zeros(len(dr))
random_close[0] = dr['close'][0]
for i in range(1, len(random_close)):
    random_close[i] = random_close[i-1] * (1 + random_chng[i])
dr['random_close'] =  random_close
dr['random_close'].plot(legend=True)

Znajdz 2 srednie, ktorych przeciecie daloby najlepsze rezultaty dla randomowego kursu

In [ ]:
maxgain = 0
for len_slow in range(10, 100, 10):
    for len_fast in range(5, 100, 5):
        if len_fast >= len_slow:
            continue

        dr['avg_fast'] = dr['random_close'].rolling(window=len_fast).mean()
        dr['avg_slow'] = dr['random_close'].rolling(window=len_slow).mean()

        positive_mask = (dr['avg_fast'] > dr['avg_slow']) 
        buy_mask = positive_mask & ~positive_mask.shift(periods=1).fillna(False)
        close_buy_mask = ~positive_mask & positive_mask.shift(periods=1).fillna(False)

        #gain %
        gain = np.sum(dr['random_close'].loc[close_buy_mask].to_numpy() / dr['random_close'].loc[buy_mask].to_numpy()[:np.sum(close_buy_mask)] - 1)
        if gain > maxgain:
            maxgain = gain
            print('New max! len_slow:%d  len_fast:%d  gain:%.2f%%  num_trans:%d' %(len_slow, len_fast, 100*gain, np.sum(close_buy_mask)))